# Testing Deferred Revenue in Python
Will this be easier for everyone to use than Matlab?

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

## Step 1: Processing Base Billings Data

In [2]:
df = pd.read_excel('../data/old/base_billings.xlsx', sheet_name='bill_DC')

In [3]:
df.sample(10)

,Document Currency,Enterprise Bu,Frequency,Invoicing Fiscal Year-Period Desc,Product Configtype ID,Revenue Recognition Category New,Rule For Bill Date,Sales Type,Subscription Term,Completed Sales Doc Currency
17012,GBP,Print & Publishing,NaN,2018-06,NaN,A,NaN,DEFERRED,1,2638.68
3048,AUD,Print & Publishing,NaN,2015-07,2Y,A,NaN,DEFERRED,0,6525.00
23008,SEK,Document Cloud,NaN,2018-10,NaN,D,YA,DEFERRED,12,360467.53
10077,EUR,Experience Cloud,1TME,2015-07,OUNIV,B,NaN,DEFERRED,0,112097.39
6480,CLP,Creative,NaN,2019-03,1Y,D,YA,DEFERRED,12,7017312.00
30614,USD,Print & Publishing,NaN,2016-01,NaN,NaN,NaN,RECOGNIZED,0,928090.32
3768,CAD,Creative,NaN,2017-09,1Y,A,NaN,DEFERRED,12,19511.72
22410,SEK,Creative,NaN,2017-09,1Y,A,NaN,DEFERRED,12,-257587.91
21368,NOK,Document Cloud,NaN,2018-03,NaN,A,NaN,DEFERRED,1,-1724.41
6907,DKK,Creative,NaN,2019-02,1Y,D,NaN,DEFERRED,12,-79050.04


## Changing the column names early since they are inconsistent across other reports

In [7]:
df.columns

Index(['curr', 'BU', 'Frequency', 'period', 'config',
       'Revenue Recognition Category New', 'rebill_rule', 'Sales Type',
       'Subscription Term', 'amount'],
      dtype='object')

In [8]:
df.rename(index = str, columns = {'Document Currency': 'curr',
                                 'Enterprise Bu': 'BU',
                                 'Invoicing Fiscal Year-Period Desc': 'period',
                                 'Product Configtype ID': 'config',
                                 'Revenue Recognition Category New': 'rev_req_type',
                                 'Rule For Bill Date': 'rebill_rule',
                                 'Completed Sales Doc Currency': 'amount'}, inplace=True)

In [9]:
df.columns

Index(['curr', 'BU', 'Frequency', 'period', 'config',
       'Revenue Recognition Category New', 'rebill_rule', 'Sales Type',
       'Subscription Term', 'amount'],
      dtype='object')

## Filter that removes any currency that has  < 10 transactions. 


In [10]:
# creates a list of the currencies and the number of transactions for each currency
vc = df['curr'].value_counts()
print(vc)

USD    7791
EUR    5291
GBP    3917
AUD    3500
JPY    3202
CHF    1678
SEK    1492
DKK    1455
NOK    1307
CAD    1109
HKD     231
RUB     199
BRL     184
NZD      38
KRW      12
SGD      12
CLP       7
COP       7
ARS       6
PEN       5
INR       1
BMD       1
MXP       1
Name: curr, dtype: int64


In [11]:
# Create variable that is true if the number of transaction is greater than 10, false otherwise
keep_these = vc.values > 10
# filtering only currencies that were greater than 10
keep_curr = vc[keep_these]
a = keep_curr.index
# filtering the dataframe to remove any of teh currencies not in our list
df = df[df['curr'].isin(a)]



## Just keeping track of the currencies we removed in our model_dict data structure

In [18]:
remove_these = vc[vc.values <= 10].index
model_dict = {'curr_removed': list(vc[remove_these].index)}
model_dict

{'curr_removed': ['CLP', 'COP', 'ARS', 'PEN', 'INR', 'BMD', 'MXP']}

In [ ]:
print("---Removing infrequent currencies from billings history---")
print('Total number of currencies in the base billings file: ', len(vc))
if sum(remove_these)==0:
    print('No currencies were removed, all contained 10 or more billings')
    print('Currencies in the base billings file')
    for item in a:
        print(a[item], end = " ")
else:
    print('\n{} Currencies were removed: '.format(sum(remove_these)))

    for item in vc[remove_these].index:
        print(item, ', ', end="")
        
    print("\n\n{} Remaining currencies: ".format(len(a)))
    for item in a:
        print(item, ', ', end="")

In [23]:
model_dict = {"curr_removed": list(vc[remove_these].index)}
model_dict

{'curr_removed': ['CLP', 'COP', 'ARS', 'PEN', 'INR', 'BMD', 'MXP']}

# Removing any of the values that are zero

In [29]:
print('This is the length of the dataframe before removing zeros: ', len(df))
df = df[df['amount']!=0]
print('This is the length of the dataframe after removing zeros: ', len(df))

This is the length of the dataframe before removing zeros:  28896
This is the length of the dataframe after removing zeros:  28896


In [30]:
df.sample(10)

,curr,BU,Frequency,period,config,Revenue Recognition Category New,rebill_rule,Sales Type,Subscription Term,amount
14476,GBP,Document Cloud,NaN,2017-05,1Y,D,YQ,DEFERRED,0,5.000000e+03
29895,USD,Experience Cloud,ONGO,2018-12,1Y,D,YH,DEFERRED,12,3.878250e+05
1292,AUD,Document Cloud,NaN,2017-09,2Y,A,NaN,DEFERRED,0,3.380500e+04
18365,JPY,Document Cloud,NaN,2015-09,NaN,NaN,NaN,RECOGNIZED,0,3.719176e+08
23947,USD,Creative,ONGO,2015-11,ONORE,D,Y1,DEFERRED,0,8.625000e+03
30539,USD,Print & Publishing,NaN,2015-09,1Y,D,YQ,DEFERRED,12,1.499500e+04
10725,EUR,Experience Cloud,ONGO,2016-09,1Y,D,Y3,DEFERRED,12,5.750000e+03
5296,CHF,Document Cloud,NaN,2015-09,1Y,D,Y3,DEFERRED,1,3.617517e+04
14149,GBP,Document Cloud,NaN,2015-09,2Y,A,NaN,DEFERRED,0,9.372000e+02
9602,EUR,Document Cloud,NaN,2017-09,NaN,D,YA,DEFERRED,12,4.184615e+05


## Clearing out the Non-Revenue billings from the file
 - No Idea what these are

In [32]:
df["Sales Type"].value_counts()

DEFERRED       22919
RECOGNIZED      5041
PRO-SVC-INV      844
Name: Sales Type, dtype: int64

In [31]:
print('Length of the dataframe before removing non-revenue billings: ', len(df))
df = df[df['Sales Type']!='NON-REV']
print('Length of the dataframe after removing non-revenue billings:  ', len(df))


Length of the dataframe before removing non-revenue billings:  28896
Length of the dataframe after removing non-revenue billings:   28804


## Starting to group the revenue by period, industry, etc

Attempting to group by the following categories
 - currency
 - period
 - sale type
 
May need to process the data differently with the deferred billings so we will start with the recognized and then the service billings

# DOING THIS ALL IN PANDAS WITH SPLIT APPLY COMBINE on Sales Type 


In [33]:
# First split the data into three dataframes
# Recognized billings
rec = df[df['Sales Type']=='RECOGNIZED']
svc = df[df['Sales Type']=='PRO-SVC-INV']
dfr = df[df['Sales Type']=='DEFERRED']

# NOW WORKING ON THE BILLINGS

### Recognized Revenue

In [ ]:
rec.sample(10)

In [35]:
# testing groupby object
gb_rec = rec.groupby(['curr', 'BU', 'period'], as_index=False).sum()

In [36]:
# the Subscription term hangs around. We are dropping that here
gb_rec.drop(labels='Subscription Term', axis=1,inplace =True)

In [37]:
gb_rec.sample(10)

,curr,BU,period,amount
1239,GBP,Other Solutions,2018-10,-5.734461e+04
1895,SEK,Experience Cloud,2017-04,1.065156e+05
1792,SEK,Creative,2016-02,3.109506e+05
2154,USD,Experience Cloud,2018-07,5.675490e+06
1194,GBP,LiveCycle,2018-03,1.755000e+03
8,AUD,Creative,2016-02,1.169964e+05
2189,USD,LiveCycle,2017-08,1.648946e+05
2285,USD,Print & Publishing,2017-12,1.414120e+07
398,CHF,Document Cloud,2018-03,3.250000e+03
1379,JPY,Document Cloud,2017-10,5.087048e+08


In [38]:
print(type(gb_rec))

<class 'pandas.core.frame.DataFrame'>


## Now doing this for the service billings

In [39]:
# testing groupby object
gb_svc = svc.groupby(['curr', 'BU', 'period'], as_index=False).sum()

In [40]:
gb_svc.drop(labels='Subscription Term', axis=1,inplace =True)
gb_svc.head(5)

,curr,BU,period,amount
0,AUD,Experience Cloud,2015-06,589499.45
1,AUD,Experience Cloud,2015-07,656493.50
2,AUD,Experience Cloud,2015-08,370780.74
3,AUD,Experience Cloud,2015-09,1251726.13
4,AUD,Experience Cloud,2015-10,291324.07


# NOW WORKING ON DEFERRED BILLINGS

## Type B billings are service agreements that will have invoices submitted before the billings are reclassified to revenue. If no invoices are assigned to the billings, the billings become revenue in 12 months

In [41]:
# filter out the type B first then do a group_by
dfr_b = dfr[dfr['Revenue Recognition Category New']=='B']

In [42]:
gb_b = dfr_b.groupby(['curr', 'BU', 'period'], as_index=False).sum()

In [43]:
gb_b.drop(labels='Subscription Term', axis=1, inplace=True)

In [44]:
gb_b.head(10)

,curr,BU,period,amount
0,AUD,Creative,2017-10,8544.00
1,AUD,Document Cloud,2017-12,3283.20
2,AUD,Document Cloud,2018-06,3425.00
3,AUD,Document Cloud,2018-12,3425.00
4,AUD,Document Cloud,2019-03,4375.00
5,AUD,Experience Cloud,2015-06,258231.69
6,AUD,Experience Cloud,2015-07,261159.25
7,AUD,Experience Cloud,2015-08,270139.50
8,AUD,Experience Cloud,2015-09,482725.30
9,AUD,Experience Cloud,2015-10,287254.60


In [45]:
print('length of deferred billings : ', len(dfr))
print('length of the type B billings: ', len(dfr_b))

length of deferred billings :  22919
length of the type B billings:  997


## Now Type A Billings
These billings are on a billing plan. The product config tells us how long before they renew

 - '2Y' = 24 months
 - '1Y' = 12 months
 - 'MTHLY' = 1 month
 
NOTE: There are also other fields in the 'Product Configtype ID' field that do not map well to a rebill period.
To fix this, we need to load up a different file and determine the length of the sales contract (type A no config)
 

In [46]:
# filtering just the type A billings
dfr_a = dfr[dfr['Revenue Recognition Category New']=='A']

In [47]:
gb_a = dfr_a.groupby(['curr', 'BU', 'period',
                     'config'], as_index=False).sum()
gb_a.drop(labels='Subscription Term', axis=1, inplace = True)

In [48]:
gb_a.sample(10)

,curr,BU,period,config,amount
1869,GBP,Document Cloud,2018-04,2Y,9035.77
388,AUD,Print & Publishing,2016-07,2Y,5823.00
1242,EUR,Document Cloud,2015-07,2Y,73537.00
3210,USD,Document Cloud,2016-11,2Y,605967.00
2393,JPY,Document Cloud,2019-02,1Y,53147395.00
2680,NOK,Document Cloud,2017-11,MTHLY,-868.00
1584,EUR,Print & Publishing,2017-03,1Y,737601.93
50,AUD,Creative,2016-10,MTHLY,-13218.71
853,CHF,Other Solutions,2016-07,OUNIV,19310.00
495,CAD,Creative,2017-10,1Y,39757.65


In [49]:
gb_a['config'].value_counts()

1Y       1856
MTHLY     823
2Y        657
OUNIV     171
OCONS      62
3Y         46
ONORE      28
Name: config, dtype: int64

### Below is just a check to see how large the billing types are across all periods

In [50]:
gb_a_config = gb_a.groupby(['config'], as_index=False).sum()
gb_a_config

,config,amount
0,1Y,6.471612e+10
1,2Y,1.973660e+08
2,3Y,8.787511e+07
3,MTHLY,-1.982427e+07
4,OCONS,2.883743e+07
5,ONORE,2.258881e+06
6,OUNIV,3.328696e+06


### These 'OCONS', 'ONORE' and 'OUNIV' data types are not actual product config IDs so we have to get them from a different data file. We are excluding these types below.

In [51]:
config_list = ['1Y', '2Y', '3Y', 'MTHLY']
test1 = gb_a['config'].isin(config_list)
sum(test1)

3382

In [52]:
test1 = gb_a[gb_a['config'].isin(config_list)]

## For now, lets just split this into gb_a_1Y, gb_a_2Y, gb_a_3y, gb_a_1M


In [53]:
gb_a_1Y = test1[test1['config']=='1Y']
gb_a_2Y = test1[test1['config']=='2Y']
gb_a_3Y = test1[test1['config']=='3Y']
gb_a_1M = test1[test1['config']=='MTHLY']

In [54]:
print('this is the lenght of type A 1M billings: ', len(gb_a_1M))
print('this is the lenght of type A 1Y billings: ', len(gb_a_1Y))
print('this is the lenght of type A 2Y billings: ', len(gb_a_2Y))
print('this is the lenght of type A 3Y billings: ', len(gb_a_3Y))

this is the lenght of type A 1M billings:  823
this is the lenght of type A 1Y billings:  1856
this is the lenght of type A 2Y billings:  657
this is the lenght of type A 3Y billings:  46


# TYPE D billings
These billings have a field 'Rule For Bill Date' that determines when new billings will occur
 - Monthly [Y1, Y2, Y3, Y5]
 - Quarterly [YQ]
 - Every 4 months [YT]
 - Semi-annual [YH]
 - Annual [YA, YC]
 - Every 2 years - [Y4]
 
 We also need to track the type D billings that do not have a 'Rule for Bill Date'

In [55]:
# for now just do a groupby on the type
# filtering just the type A billings
dfr_d = dfr[dfr['Revenue Recognition Category New']=='D']

In [56]:
gb_d = dfr_d.groupby(['curr', 'BU', 'period',
                     'rebill_rule'], as_index=False).sum()
gb_d.drop(labels='Subscription Term', axis=1, inplace = True)

In [57]:
gb_d_mthly = gb_d[gb_d['rebill_rule'].isin(['Y1', 'Y2', 'Y3', 'Y5'])]
gb_d_qtrly = gb_d[gb_d['rebill_rule']=='YQ']
gb_d_four_mths = gb_d[gb_d['rebill_rule']=='YT']
gb_d_semi_ann = gb_d[gb_d['rebill_rule']=='YH']
gb_d_annual = gb_d[gb_d['rebill_rule'].isin(['YA', 'YC'])]
gb_d_two_yrs = gb_d[gb_d['rebill_rule']=='Y4']

In [58]:
gb_d['rebill_rule'].value_counts()

YA    1630
Y3    1438
YQ     530
Y1     256
YC     246
YH     243
Y2      95
Y4      23
Y7      18
YT       8
BT       1
Name: rebill_rule, dtype: int64

### Note: There is one type D billings that is listed as 'BT' I don't know what this means, but it was a $180 EUR payment from January 2017, so we will ignore

In [59]:
print('Length of monthly', len(gb_d_mthly))
print('Length of quarterly', len(gb_d_qtrly))
print('Length of four months', len(gb_d_four_mths))
print('Length of semi ann', len(gb_d_semi_ann))
print('Length of annual', len(gb_d_annual))
print('Length of two years', len(gb_d_two_yrs))

Length of monthly 1789
Length of quarterly 530
Length of four months 8
Length of semi ann 243
Length of annual 1876
Length of two years 23


In [60]:
what_is_YT = gb_d[gb_d['rebill_rule']=='YT']
what_is_YT.head(10)

,curr,BU,period,rebill_rule,amount
4054,USD,Experience Cloud,2015-06,YT,11666.60
4068,USD,Experience Cloud,2015-08,YT,549861.11
4081,USD,Experience Cloud,2015-10,YT,11666.60
4101,USD,Experience Cloud,2016-01,YT,612916.60
4106,USD,Experience Cloud,2016-02,YT,-601250.00
4166,USD,Experience Cloud,2016-12,YT,13440.00
4190,USD,Experience Cloud,2017-04,YT,13440.00
4214,USD,Experience Cloud,2017-08,YT,13440.00


## Note: There are only 8 payments (back from 2017) that paid on a 4 month basis. 
We will ignore these as well.

## NOW WE NEED TO BUILD A DATAFRAME THAT INTEGRATES THIS DATA

- We will have the following descriptive fields
   - Invoicing Fiscal Year-Period
   - Document Currency
   - Enterprise BU

- We will have the following fields based on rebilling rule
   - Recognized
   - Service
   - Monthly
   - Quarterly
   - Annual
   - Two Years
   - Three Years

In [61]:
# We need to do it this way when we get to a .py file!
list_df = [gb_rec, gb_svc, gb_b,
        gb_a_1M,    gb_a_1Y,    gb_a_2Y,       gb_a_3Y, 
        gb_d_mthly, gb_d_qtrly, gb_d_semi_ann, gb_d_annual, gb_d_two_yrs]

list_columns = ['recognized', 'service', 'deferred_B', 
    'deferred_1M_a', 'deferred_1Y_a', 'deferred_2Y_a', 'deferred_3Y_a',
    'deferred_1M_d', 'deferred_3M_d', 'deferred_6M_d', 'deferred_1Y_d', 'deferred_2Y_d']


In [62]:
def merge_new_dataframe(old_df, new_df, new_column):
    df_merged = pd.merge(old_df, new_df, how='outer', 
                     left_on=['curr', 'BU', 'period'],
                    right_on=['curr', 'BU', 'period'])
    df_merged.rename(index=str, columns={'amount': new_column}, inplace=True)
    
    #need to drop the product configtype id for merges where the new_df is of type A
    config_str = 'config'
    rule_str = 'rebill_rule'
    if config_str in df_merged.columns:
        df_merged.drop(columns=['config'], inplace=True)
    
    if rule_str in df_merged.columns:
        df_merged.drop(columns=['rebill_rule'], inplace=True)
        
    return df_merged

In [63]:
def merge_all_dataframes(list_df, list_columns):
    for i, df in enumerate(list_df):
        print('This is i:', i)
        #print("This is the df: ", df.head())
        print('referencing the column: ', list_columns[i])

        if i==0:
            df_merged = list_df[0]
            df_merged.rename(index=str, columns={'amount': list_columns[i]}, inplace=True)
        else:
            df_merged = merge_new_dataframe(df_merged, df, list_columns[i])

    return df_merged
    

In [64]:
df = merge_all_dataframes(list_df, list_columns)

This is i: 0
referencing the column:  recognized
This is i: 1
referencing the column:  service
This is i: 2
referencing the column:  deferred_B
This is i: 3
referencing the column:  deferred_1M_a
This is i: 4
referencing the column:  deferred_1Y_a
This is i: 5
referencing the column:  deferred_2Y_a
This is i: 6
referencing the column:  deferred_3Y_a
This is i: 7
referencing the column:  deferred_1M_d
This is i: 8
referencing the column:  deferred_3M_d
This is i: 9
referencing the column:  deferred_6M_d
This is i: 10
referencing the column:  deferred_1Y_d
This is i: 11
referencing the column:  deferred_2Y_d


In [65]:
df.sample(10)

,curr,BU,period,recognized,service,deferred_B,deferred_1M_a,deferred_1Y_a,deferred_2Y_a,deferred_3Y_a,deferred_1M_d,deferred_3M_d,deferred_6M_d,deferred_1Y_d,deferred_2Y_d
3088,CAD,Print & Publishing,2019-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25285.00,NaN,72455.82,NaN
1704,JPY,Experience Cloud,2018-03,1.040093e+08,1.136591e+08,1.526175e+08,NaN,37039454.00,NaN,NaN,5543510.00,2150000.00,44086130.00,1102110.00,NaN
1860,NOK,Creative,2017-06,2.070260e+05,NaN,NaN,-20907.57,-27144.35,NaN,NaN,12850355.85,NaN,NaN,2751043.19,NaN
473,CHF,Experience Cloud,2017-11,1.874013e+04,7.510130e+04,7.632000e+03,NaN,814599.74,NaN,NaN,NaN,NaN,NaN,-49884.58,NaN
2722,USD,LiveCycle,2016-02,2.669305e+05,NaN,NaN,NaN,762883.51,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1658,JPY,Document Cloud,2018-08,3.011405e+08,NaN,NaN,-72140.00,49776511.00,499200.0,3067344.0,14763949.00,NaN,NaN,26147513.00,NaN
1870,NOK,Creative,2018-04,1.767270e+05,NaN,NaN,-14486.43,-63695.86,NaN,NaN,11931680.07,NaN,NaN,3527567.30,NaN
904,EUR,Experience Cloud,2016-01,5.903991e+06,1.110991e+06,4.727292e+05,NaN,2741852.10,NaN,NaN,66619.39,1253508.55,122427.07,9368041.03,NaN
76,AUD,Document Cloud,2016-11,4.746648e+05,NaN,NaN,-330.89,552417.79,27285.0,NaN,244892.70,NaN,NaN,775620.80,NaN
161,AUD,Experience Cloud,2018-11,2.580769e+06,8.208899e+05,2.609291e+05,NaN,182227.02,NaN,NaN,14253.71,1130113.59,NaN,1360352.24,NaN


In [66]:
def clean_df_columns(df):
    
    # clean up NaNs before adding 
    df = df.fillna(value=0)
    
    # Monthly
    df['deferred_1M'] = df['deferred_1M_a']+df['deferred_1M_d']
    df.drop(labels=['deferred_1M_a', 'deferred_1M_d'], axis=1, inplace=True)
    
    # Annual
    df['deferred_1Y'] = df['deferred_1Y_a']+df['deferred_1Y_d']
    df.drop(labels=['deferred_1Y_a', 'deferred_1Y_d'], axis=1, inplace=True)
    
    # Two-Year
    df['deferred_2Y'] = df['deferred_2Y_a']+df['deferred_2Y_d']
    df.drop(labels=['deferred_2Y_a', 'deferred_2Y_d'], axis=1, inplace=True)
    
    # renaming 3Y, 3M and 6M
    df.rename(index=str, columns = {'deferred_3Y_a':'deferred_3Y', 'deferred_3M_d':'deferred_3M', 
                               'deferred_6M_d': 'deferred_6M'}, inplace=True)

    #cleaning up the longer column names
    df.rename(index=str, columns = {'curr': 'curr',
                               'BU':'BU',
                               'period':'period'}, inplace=True)
    
    return df

In [67]:
df = clean_df_columns(df)

In [68]:
df.sample(10)

,curr,BU,period,recognized,service,deferred_B,deferred_3Y,deferred_3M,deferred_6M,deferred_1M,deferred_1Y,deferred_2Y
1867,NOK,Creative,2018-01,2.108570e+05,0.0,0.0,0.0,0.00,0.0,1.142924e+07,2.637839e+06,0.00
3088,CAD,Print & Publishing,2019-03,0.000000e+00,0.0,0.0,0.0,25285.00,0.0,0.000000e+00,7.245582e+04,0.00
1760,JPY,Other Solutions,2016-07,-4.789490e+05,0.0,0.0,0.0,0.00,0.0,0.000000e+00,0.000000e+00,0.00
3058,BRL,Document Cloud,2019-03,0.000000e+00,0.0,0.0,0.0,0.00,0.0,8.223903e+04,1.673160e+05,0.00
337,CAD,LiveCycle,2017-11,6.409000e+02,0.0,0.0,0.0,0.00,0.0,0.000000e+00,0.000000e+00,0.00
1636,JPY,Document Cloud,2017-06,3.980026e+08,0.0,0.0,0.0,0.00,0.0,1.567488e+07,1.156652e+08,0.00
1996,NOK,Print & Publishing,2016-02,3.348000e+04,0.0,0.0,0.0,0.00,0.0,1.057500e+04,7.550000e+02,0.00
1430,GBP,Other Solutions,2018-04,-6.940700e+02,0.0,0.0,0.0,0.00,0.0,0.000000e+00,0.000000e+00,0.00
2354,USD,Creative,2016-04,4.028866e+06,725.0,0.0,0.0,421272.92,409825.0,6.289047e+07,3.780630e+07,62595.56
1559,JPY,Creative,2017-04,8.395470e+07,0.0,0.0,0.0,0.00,0.0,1.035968e+09,1.455778e+09,1410500.00


In [69]:
# Make this a function to be cleaned up somehow
del dfr
del dfr_a
del dfr_b
del dfr_d
del gb_a
del gb_a_1M
del gb_a_1Y
del gb_a_2Y
del gb_a_3Y
del gb_b, 
del gb_d
del gb_svc, gb_rec, gb_d_two_yrs
del gb_d_four_mths, gb_d_qtrly, gb_d_semi_ann


# Now working on the ZCC billings

These billings are type D billings that did not populate the rebill_rule field of the database.

They have a 'sales document type' = 'ZCC"

The billings themselves are being created from a tableau report that looks for additions to the deferred revenue waterfall based on billings of type D and have a sales document type of ZCC

In [ ]:
whos

In [70]:
df.sample(10)

,curr,BU,period,recognized,service,deferred_B,deferred_3Y,deferred_3M,deferred_6M,deferred_1M,deferred_1Y,deferred_2Y
875,EUR,Document Cloud,2018-02,2263908.82,0.00,0.00,0.0,53250.00,0.00,325277.11,5000248.65,75750.00
1216,GBP,Document Cloud,2015-08,671437.19,0.00,0.00,0.0,0.00,0.00,185386.68,872515.44,4453.92
476,CHF,Experience Cloud,2018-02,49329.66,73168.80,57887.36,0.0,0.00,0.00,0.00,1013799.79,0.00
2772,USD,Other Solutions,2016-06,-55971.53,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00
1949,NOK,Experience Cloud,2017-07,191015.51,209899.63,62799.00,0.0,250228.25,0.00,0.00,0.00,0.00
543,CHF,Print & Publishing,2016-05,5033.00,0.00,0.00,0.0,0.00,0.00,2708.72,0.00,0.00
3068,HKD,Document Cloud,2018-12,0.00,0.00,0.00,0.0,0.00,0.00,32345.00,34350.00,0.00
3048,NOK,Print & Publishing,2018-06,0.00,0.00,0.00,0.0,0.00,0.00,22359.00,283.00,0.00
1204,GBP,Creative,2018-06,190684.65,0.00,0.00,0.0,0.00,2124.99,15387658.55,8915362.28,14640.59
447,CHF,Experience Cloud,2015-09,423617.15,275522.17,49417.40,0.0,0.00,0.00,0.00,54853.57,0.00


# TO BE DONE:

1. Clean up the type F billings (at least check to see if they are necessary)
2. Make a function to delete all intermediate dataframes
3. Add type A no config function
4. Add type D ZCC billings

5. Work on the forecast part of this

6. Load up FX rates

In [ ]:
# Adobe financial calendar
df_cal = pd.read_excel('../data/old/ADOBE_FINANCIAL_CALENDAR.xlsx', 'ADBE_cal')

In [ ]:
df_cal.head()

## ZCC Billings

In [ ]:
df_ZCC = pd.read_excel('../data/old/type_D_ZCC_billings.xlsx', sheet_name='DC')

In [ ]:
df_ZCC.sample(10)

### Renaming the columns

In [ ]:
df_ZCC.columns

In [ ]:
df_ZCC.rename(index = str, columns = {'Document Currency': 'curr',
                                      'Enterprise BU Description':'BU',
                                      'Rule for Bill Date Code': 'rebill_rule',
                                      'Week of FICA Posting Date (YYYYMMDD) (copy)': 'fiscal_week',
                                      'DF Additions - Doc Curr': 'amount'}, inplace = True)

In [ ]:
df_ZCC.columns

### Checking that we do not have any currencies that need to be removed


In [ ]:
df_ZCC['curr'].value_counts()

In [ ]:
model_dict

In [ ]:
ZCC_curr = df_ZCC['curr'].unique()

In [ ]:
for item in ZCC_curr:
    if item in model_dict['curr_removed']:
        print('This currency needs to be removed: ', item)

There are no currencies that need to be removed

# Now we are clearing out nans

In [ ]:
print('Number of NaNs to be cleared out: ', sum(df_ZCC['amount'].isna()))
ZCC_isna = df_ZCC['amount'].isna()
print("This is the length of the ZCC records before clearing NAs: ", len(df_ZCC))
df_ZCC = df_ZCC[~ZCC_isna]
print("This is the length of the ZCC records before clearing NAs: ", len(df_ZCC))

In [ ]:
df_cal.sample(5)

In [ ]:
df_ZCC.sample(10)

In [ ]:
def add_ZCC_periods(df_ZCC, df_cal):
    ''' 
    Takes each ZCC period billing and maps it to the fiscal calendar then assignes the
    ZCC billing to the dataframe df
    '''
    period_list = []
    
    for i in range(len(df_ZCC)):
        # this_date is the date we need mapped to a period
        this_date = df_ZCC['fiscal_week'].iloc[i]
        
        #max index will be the index to the period in our financial calendar
        this_index = df_cal['Per_End']<=this_date
        max_index = sum(this_index)

        this_year = df_cal['Year'].iloc[max_index].astype(str)
        this_period= df_cal['Period'].iloc[max_index].astype(str)

        # formatting the period string from the calendar
        if len(this_period)== 1:
            this_period = '0'+this_period

        period_ticker = this_year + '-' + this_period

        period_list.append(period_ticker)

    df_ZCC['period'] = period_list
    return df_ZCC


In [ ]:
df_ZCC.head()

In [ ]:
df_ZCC = add_ZCC_periods(df_ZCC, df_cal)

In [ ]:
df_ZCC.tail()

## Now we need to determine the length of the ZCC billings to and add this to our dataframe

In [ ]:
df.columns

In [ ]:
df_ZCC.columns

In [ ]:
gb_ZCC = df_ZCC.groupby(['curr', 'BU', 'period',
                     'rebill_rule'], as_index=False).sum()
#gb_d.drop(labels='Subscription Term', axis=1, inplace = True)

In [ ]:
gb_ZCC.sample(20)

In [ ]:
gb_ZCC['rebill_rule'].value_counts()

In [ ]:
gb_ZCC_mthly = gb_ZCC[gb_ZCC['rebill_rule'].isin(['Y1', 'Y2', 'Y3', 'Y5'])]
gb_ZCC_qtrly = gb_ZCC[gb_ZCC['rebill_rule']=='YQ']
gb_ZCC_four_mths = gb_ZCC[gb_ZCC['rebill_rule']=='YT']
gb_ZCC_semi_ann = gb_ZCC[gb_ZCC['rebill_rule']=='YH']
gb_ZCC_annual = gb_ZCC[gb_ZCC['rebill_rule'].isin(['YA', 'YC'])]
gb_ZCC_two_yrs = gb_ZCC[gb_ZCC['rebill_rule']=='Y4']

There are no four month billings

In [ ]:
len(gb_ZCC_four_mths)

There are two YX billings for small amounts that I do not have mapped to a frequency.
They will be ignored

In [ ]:
what_is_YX = gb_ZCC[gb_ZCC['rebill_rule']=='YX']
what_is_YX.head(10)

## I need to merge the ZCC billings and then clean up the columns. 
 


In [ ]:
# We need to do it this way when we get to a .py file!
list_ZCC_df = [gb_ZCC_mthly, gb_ZCC_qtrly, gb_ZCC_semi_ann,
           gb_ZCC_annual, gb_ZCC_two_yrs]

list_ZCC_columns = ['deferred_1M', 'deferred_3M', 'deferred_6M', 
    'deferred_1Y', 'deferred_2Y']


In [ ]:
df_ZCC_merged = merge_all_dataframes(list_ZCC_df, list_ZCC_columns)

In [ ]:
df_ZCC_merged.sample(10)

### Clearing out any zeros

In [ ]:
df_ZCC_merged.drop(columns=['rebill_rule_x', 'rebill_rule_y'], inplace=True)
df_ZCC_merged.fillna(0, inplace=True)
df_ZCC_merged.sample(10)

In [ ]:
df.columns

In [ ]:
df_cal.sample(5)

In [ ]:
test_merge = pd.merge(df, df_ZCC_merged, how='outer', 
                 left_on=['curr', 'BU', 'period'],
                right_on=['curr', 'BU', 'period'])
#test_merge.rename(index=str, columns={'amount': new_column}, inplace=True)


In [ ]:
test_merge

In [ ]:

# clean up NaNs before adding 
test_merge = test_merge.fillna(value=0)

# Monthly
test_merge['deferred_1M'] = test_merge['deferred_1M_x']+test_merge['deferred_1M_y']
test_merge.drop(labels=['deferred_1M_x', 'deferred_1M_x'], axis=1, inplace=True)

# Quarterly
test_merge['deferred_3M'] = test_merge['deferred_3M_x']+test_merge['deferred_3M_y']
test_merge.drop(labels=['deferred_3M_x', 'deferred_3M_y'], axis=1, inplace=True)

# Semi-Annual
test_merge['deferred_6M'] = test_merge['deferred_6M_x']+test_merge['deferred_6M_y']
test_merge.drop(labels=['deferred_6M_x', 'deferred_6M_y'], axis=1, inplace=True)

# Annual
test_merge['deferred_1Y'] = test_merge['deferred_1Y_x']+test_merge['deferred_1Y_y']
test_merge.drop(labels=['deferred_1Y_x', 'deferred_1Y_y'], axis=1, inplace=True)

# 2-Years
test_merge['deferred_2Y'] = test_merge['deferred_2Y_x']+test_merge['deferred_2Y_y']
test_merge.drop(labels=['deferred_2Y_x', 'deferred_2Y_y'], axis=1, inplace=True)





In [ ]:
test_merge.columns

In [ ]:
test_merge.sample(20)

In [ ]:
## Looks like the type ZCC billings are complete

In [ ]:
# renaming the main dataframe as df
df = test_merge.copy()

# Type A No Config Type Billings

This file contains type A billings that have a revenue contract start date and end date. We need to map these into the terms of our dataframe.

### Steps:
1. Rename the columns
2. This file has entries for pennies. Need to clear out anything less than $10 in absolute value
3. Determine the length of time between start date and end date
4. Group this dataframe by currency, period and BU
5. Merge this final dataframe with the larger dataframe

## NOTE: This file contains two different start date and end date columns. We need to look at all of them

In [ ]:
df_A = pd.read_excel('../data/old/type_A_no_config.xlsx', 'DC')

In [ ]:
df_A.sample(10)

In [ ]:
df_A.columns

In [ ]:
df_A.rename(index=str, columns={'Document Currency':'currency', 
                               'Enterprise Bu':'BU',
                               'Invoicing Fiscal Year-Period Desc':'period',
                               'Rev Rec Contract End Date Hdr':'end_date_1',
                               'Rev Rec Contract End Date Item':'end_date_2',
                               'Rev Rec Contract Start Date Hdr': 'start_date_1',
                               'Rev Rec Contract Start Date Item': 'start_date_2',
                               'Completed Sales Doc Currency':'amount'}, inplace=True)

df_A.drop(columns='Product Configtype ID', inplace=True)

In [ ]:
df_A.columns

### Dealing with the duplicate dates by taking a max

In [ ]:
df_A.head()

In [ ]:
df_A['start_date_str'] = df_A[['start_date_1','start_date_2']].max(axis=1).astype(int).astype(str)
df_A['end_date_str'] = df_A[['end_date_1','end_date_2']].max(axis=1).astype(int).astype(str)

In [ ]:
df_A.sample(10)

In [ ]:
df_A['start_date'] = pd.to_datetime(df_A['start_date_str'])
df_A['end_date'] = pd.to_datetime(df_A['end_date_str'])

In [ ]:
df_A.drop(labels=['end_date_1', 'end_date_2', 'start_date_1', 'start_date_2',
                  'start_date_str', 'end_date_str'], axis=1, inplace=True)

In [ ]:
df_A['month_interval']=(df_A['end_date']-df_A['start_date'])

In [ ]:
df_A.sample(10)

In [ ]:
df_A['months']= (df_A['month_interval']/ np.timedelta64(1,'M')).round(0)

### Now I need to map the months into the different integers in my dataframe


In [ ]:
list_rebills = [1, 3, 6, 12, 24, 36]
temp_rebill = np.zeros_like(df_A['months'])
for i in range(len(df_A)):
    temp_rebill[i] = min(list_rebills, key=lambda x:abs(x-df_A['months'][i]))
df_A['rebill_months']=temp_rebill


In [ ]:
df_A.sample(10)

In [ ]:
plt.scatter(df_A['months'], df_A['rebill_months'])

## Grouping the dataframe by rebill_months

In [ ]:
# drop what we dont need
df_A.drop(columns = ['start_date', 'end_date', 'month_interval', 'months'], axis=1, inplace=True)

In [ ]:
df_A.head()

In [ ]:
#gb_A = df+A.groupby(['curr', 'BU', 'period'], as_index=False).sum()

In [ ]:
#medals = df.pivot_table('no of medals', ['Year', 'Country'], 'medal')
temp = df_A.pivot_table('amount', ['currency', 'BU', 'period'], 'rebill_months')

In [ ]:
temp

In [ ]:
temp.columns[5]

In [ ]:
temp.fillna(0)

In [ ]:
temp.rename(index=str, columns = {''})

In [ ]:
whos